In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
from p_tqdm import p_map
from itertools import product
import os
from pathlib import Path
from p_tqdm import p_map
from functools import partial

In [ ]:
recipes = ['adding_up', 'risk_aversion']
discs = ['constant', 'euler_ramsey']
sectors = ["CAMEL_clipped"] # ["coastal", "labor_clipped", "agriculture_clipped", 'AMEL_clipped', "mortality_clipped", "energy_clipped" ] # "CAMEL_clipped", 
mask_names = ["gdppc_emissions"] # ["climate","emissions","gdppc","gdppc_emissions_climate"]
mask_vars =[ f'q{i[0]}_q{i[1]}' for i in [
    # [0.001, 0.999],
    # [0.005, 0.995],
    [0.01, 0.99],
    # [0.05, 0.95],
]]

# years=[2020]
years = range(2020,2081,10)
etas =  [1.567899395, 2.0, 1.016010255, 1.244459066, 1.421158116,]
eta_rhos = {
    2.0 : 0.0,
    1.016010255 : 9.149608e-05,
    1.244459066 : 0.00197263997,
    1.421158116 : 0.00461878399,
    1.567899395 : 0.00770271076
}

In [ ]:
def resave_file(args):

    sector, year, recipe, disc, eta, mask_name, mask_var = args
    
    indir=f"/mnt/CIL_integration/gas_test/['CH4', 'CO2_Fossil', 'N2O']/{sector}/{year}"

    ds = xr.open_dataset(
        f"{indir}/unmasked_None/{recipe}_{disc}_eta{eta}_rho{eta_rhos[eta]}_uncollapsed_sccs.nc4"
    )

    if mask_name != 'unmasked':
        mask = xr.open_dataset(f"/shares/gcp/integration/rff/climate/masks/CO2_Fossil/{mask_name}_based_masks.nc4")[mask_var]
        ds = ds.where(mask, drop=True)
    else:
        mask_name = "unmasked"
        mask_var = "None"

    # mask and put back together
    outpath =  f"{indir}/{mask_name}_{mask_var}/"
    os.makedirs(outpath, exist_ok = True)
    print(f"{outpath}/{recipe}_{disc}_eta{eta}_rho{eta_rhos[eta]}_uncollapsed_sccs.nc4")
    ds.to_netcdf(f"{outpath}/{recipe}_{disc}_eta{eta}_rho{eta_rhos[eta]}_uncollapsed_sccs.nc4")
    ds.close()


In [ ]:
p_map(resave_file,
      list(product(sectors, years, recipes, discs, etas, mask_names, mask_vars)),
      num_cpus=30
     )